In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

# 1. Load the data
df = pd.read_csv('Cleaned_Indian_Food_Dataset.csv')  # replace with your actual file path
df = df[['TranslatedInstructions', 'Cuisine']].dropna()

# 2. Encode labels
label_encoder = LabelEncoder()
df['cuisine_label'] = label_encoder.fit_transform(df['Cuisine'])

# 3. Train/test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['TranslatedInstructions'].tolist(), df['cuisine_label'].tolist(), test_size=0.2, random_state=42
)

# 4. Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 5. Create Dataset class
class RecipeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[idx])
        }

train_dataset = RecipeDataset(train_texts, train_labels, tokenizer)
val_dataset = RecipeDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 6. Define the model
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        out = self.dropout(pooled_output)
        return self.fc(out)

num_classes = len(label_encoder.classes_)
model = BERTClassifier(num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 7. Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for batch in tqdm(loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# 8. Evaluation
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

all_labels = list(range(len(label_encoder.classes_)))
# 9. Train the model
for epoch in range(25):  # You can change the number of epochs
    print(f"\nEpoch {epoch+1}")
    train_loss = train_epoch(model, train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    preds, truths = evaluate(model, val_loader)
    # print(classification_report(truths, preds, target_names=label_encoder.classes_))
    print(classification_report(
    truths,
    preds,
    labels=all_labels,
    target_names=label_encoder.classes_,
    zero_division=0  # to avoid division by zero errors
    ))

torch.save(model.state_dict(), 'bert_recipe_classifier.pt')

# Optionally, save the label encoder as well
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

                          precision    recall  f1-score   support

                  Afghan       0.00      0.00      0.00         0
                 African       0.50      1.00      0.67         3
                American       0.00      0.00      0.00         0
                  Andhra       0.85      0.85      0.85        20
               Appetizer       0.00      0.00      0.00         1
                    Arab       0.00      0.00      0.00         3
                   Asian       0.22      0.17      0.19        12
                Assamese       1.00      0.50      0.67         2
                  Awadhi       0.47      0.70      0.56        10
         Bengali Recipes       0.82      0.69      0.75        39
                  Bihari       0.75      1.00      0.86         3
                 British       0.00      0.00      0.00         3
                  Brunch       0.00      0.00      0.00         0
                 Burmese       0.00      0.00      0.00         0
         

100%|██████████| 297/297 [02:06<00:00,  2.35it/s]


Training Loss: 0.0594


100%|██████████| 75/75 [00:12<00:00,  6.05it/s]


                          precision    recall  f1-score   support

                  Afghan       0.00      0.00      0.00         0
                 African       0.50      1.00      0.67         3
                American       0.00      0.00      0.00         0
                  Andhra       0.89      0.80      0.84        20
               Appetizer       0.00      0.00      0.00         1
                    Arab       0.00      0.00      0.00         3
                   Asian       0.14      0.17      0.15        12
                Assamese       1.00      0.50      0.67         2
                  Awadhi       0.58      0.70      0.64        10
         Bengali Recipes       0.85      0.74      0.79        39
                  Bihari       0.67      0.67      0.67         3
                 British       0.00      0.00      0.00         3
                  Brunch       0.00      0.00      0.00         0
                 Burmese       0.00      0.00      0.00         0
         